# Задание 1.2: Модель эластичности

In [1]:
from typing import List, Dict, Any, Tuple, Union, Optional

from datetime import datetime, timedelta
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")

import logging

logging.basicConfig()
logger = logging.getLogger("model")
logger.setLevel(logging.INFO)

## Данные

In [2]:
sales_df = pd.read_parquet("./hm/sales.parquet")
sales_df["ds"] = sales_df["ds"].astype(str)
sales_df.head()

,sku_id,gmv,orders_num,price,ds
283,284,121161.0,116,1044.49,20230601
302,303,10413.0,186,55.98,20230601
235,236,550034.0,2569,214.10,20230601
206,207,18895.0,250,75.58,20230601
245,246,264666.0,3232,81.89,20230601


## Модель эластичности

In [3]:
def calculate_elasticity(df: pd.DataFrame, column: str, lvl: List[str]) -> pd.DataFrame:
    df[f"prev_{column}"] = df.groupby("sku_id")[column].shift(1)
    df["prev_price"] = df.groupby("sku_id")["price"].shift(1)
    df = df[df["prev_price"] - df["price"] != 0]
    df[f"elasticity_{column}"] = (
        (df[column] - df[f"prev_{column}"]) / (df["price"] - df["prev_price"])
    ) * ((df["price"] + df["prev_price"]) / (df[column] + df[f"prev_{column}"]))
    df = df.groupby(lvl).agg({f"elasticity_{column}": "mean"}).reset_index()
    df[f"elasticity_{column}"] = df[f"elasticity_{column}"].clip(upper=-0.01, lower=-3)
    return df

In [4]:
elasticity_gmv_df = calculate_elasticity(
    df=sales_df.copy(deep=True), column="gmv", lvl=["sku_id"]
)
elasticity_gmv_df.head()

,sku_id,elasticity_gmv
0,1,-0.01
1,3,-0.01
2,4,-0.01
3,7,-0.01
4,8,-0.01


In [5]:
elasticity_orders_num_df = calculate_elasticity(
    df=sales_df, column="orders_num", lvl=["sku_id"]
)
elasticity_orders_num_df.head()

,sku_id,elasticity_orders_num
0,1,-0.091652
1,3,-0.389979
2,4,-0.010000
3,7,-0.010000
4,8,-0.010000


In [6]:
df = elasticity_gmv_df.merge(elasticity_orders_num_df, on="sku_id", how="inner")
df.head()

,sku_id,elasticity_gmv,elasticity_orders_num
0,1,-0.01,-0.091652
1,3,-0.01,-0.389979
2,4,-0.01,-0.010000
3,7,-0.01,-0.010000
4,8,-0.01,-0.010000


In [7]:
df.to_csv("./to_karp_5_2.csv", index=False)